1. Parsing the System of Equations (1 point)

In [3]:
import pathlib
var=("x","y","z")
def coef(token: str)->float:
    num= token[:-1]
    return float(num) if num else 1.0

def parse_line(line: str) -> tuple[list[float], float]:
    tokens=line.strip().split()
    coeff={}
    coeff={v:0.0 for v in var}
    coeff["x"]=coef(tokens[0])
    if(tokens[1]=="+"):
         coeff["y"]=coef(tokens[2]) 
    else:
         coeff["y"]=-coef(tokens[2])
    if(tokens[3]=="+"):
         coeff["z"]=coef(tokens[4])
    else:
         coeff["z"]=-coef(tokens[4])
    
    if(tokens[5]!="="):
        raise ValueError("Invalid format")
    b=float(tokens[6])
    print(coeff)
    return [coeff[v] for v in var], b
        

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    A: list[list[float]] = []  
    B: list[float] = [] 
    with path.open("r",encoding="utf-8") as f:
        for raw in f:
            line=raw.strip()
            if not line:
                continue
            row, b=parse_line(line)
            A.append(row)
            B.append(b)
    return A, B
                   

A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

{'x': 2.0, 'y': 3.0, 'z': -1.0}
{'x': 1.0, 'y': -1.0, 'z': 4.0}
{'x': 3.0, 'y': 1.0, 'z': 2.0}
A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]



2. Matrix and Vector Operations (5 points)

2.1. Determinant


In [4]:
def determinant(matrix: list[list[float]]) -> float:
    det=0.0
    if(len(matrix[0])==2): 
        det= matrix[0][0]*matrix[1][1]-matrix[0][1]*matrix[1][0]
    if(len(matrix[0])==3):
        det= (matrix[0][0] * (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1])
            - matrix[0][1] * (matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0])
            + matrix[0][2] * (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix[2][0]))
    return det

print(f"{determinant(A)=}")

determinant(A)=14.0


2.2. Trace

In [5]:
def trace(matrix: list[list[float]]) -> float:
    return matrix[0][0] + matrix[1][1] + matrix[2][2]
print(f"{trace(A)=}")

trace(A)=3.0


2.3. Vector norm

In [6]:
from math import sqrt
def norm(vector: list[float]) -> float:
    return sqrt(vector[0]**2+vector[1]**2+vector[2]**2)
print(f"{norm(B)=}")

norm(B)=10.488088481701515


2.4. Transpose of matrix

In [7]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    matrix2=[[0.0 for _ in range(3)] for _ in range(3)]
    for i in range(3):
       for j in range(3):
            matrix2[i][j]=matrix[j][i]
    return matrix2

print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


2.5. Matrix-vector multiplication

In [8]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    matrix2=[0.0 for _ in range(3)]
    for i in range(3):
        for j in range(3):
            matrix2[i]+=matrix[i][j]*vector[j]
    return matrix2
            
print(f"{multiply(A,B)=}")
    

multiply(A,B)=[21.0, 27.0, 35.0]


3. Solving using Cramer's Rule (1 point)

In [9]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    detA=determinant(matrix)
    print(f"{detA=}")
    if detA==0:
        raise ValueError("The system has no unique solution")
    solution=[0.0 for _ in range(3)]
    for i in range(3):
        matrix2=[[matrix[r][c] for c in range(3)] for r in range(3)]
        for r in range(3):
            matrix2[r][i]=vector[r]
        detAi=determinant(matrix2)
        solution[i]=detAi/detA
    return solution
print(f"{solve_cramer(A, B)=}")

detA=14.0
solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


4. Solving using Inversion (3 points)

In [10]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    return [[matrix[r][c] for c in range(3) if c != j] for r in range(3) if r != i]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactor_matrix=[[0.0 for _ in range(3)] for _ in range(3)]
    for i in range(3):
        for j in range(3):
            minor_ij=minor(matrix,i,j)
            cofactor_matrix[i][j]=(((-1)**(i+j))*determinant(minor_ij))
    return cofactor_matrix
            

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    cofactor_matrix=cofactor(matrix)
    print(f"{cofactor_matrix=}")
    return transpose(cofactor_matrix)

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    matrix2=[[0.0 for _ in range(3)] for _ in range(3)]
    adjoint_matrix=adjoint(matrix)
    detA=determinant(matrix)
    print(f"{detA=}")
    for i in range(3):
        for j in range(3):
            matrix2[i][j]=matrix[i][j]
    for i in range(3):
        for j in range(3):
            matrix2[i][j]=adjoint_matrix[i][j]*(1/detA)   
    solution= multiply(matrix2,vector)
    return solution

print(A)
print(B)
print(f"{solve(A, B)=}")

[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]]
[5.0, 6.0, 7.0]
cofactor_matrix=[[-6.0, 10.0, 4.0], [-7.0, 7.0, 7.0], [11.0, -9.0, -5.0]]
detA=14.0
solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714293]
